In [1]:
import pandas as pd
import as np

In [2]:
adi = pd.read_csv('full_cleaned_data.tsv', sep='\t')

In [3]:
adi.head()

,Unnamed: 0,text,split,source,dialect,country,num_arabic_chars
0,1151,هه اصلا حاليا فى وفد قطري موجود بغزه واللعبه ه...,train,SHAMI,LEV,PL,51
1,1152,عادي طعميها لأنها ما راح تكتفي من صدرك صار بده...,train,SHAMI,LEV,PL,48
2,1153,اعملي اللي عليكي لوجه الله,train,SHAMI,LEV,PL,22
3,1154,حكت شفتها اكتر من مره بلبس قصير واكيد زوجي شافها,train,SHAMI,LEV,PL,39
4,1155,و هيا وإياه بتزوجو هه,train,SHAMI,LEV,PL,17
